# K Means to cluster the incidents into 4 groups (Priority levels)

The Incident Management dataset has about 141712 records of 24918 incidents. Each state of the incident is being captured as an individual record with few exceptions where the closed state of an incident is recorded more than once. With the help of the below segment of the code, we load and clean the Incident Management data so that only one record representing the truly closed state per incident is obtained.

------------------------------------------------------------------------------------------------------------------------------

##### Create a spark session and load the Incident Management Data set

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('IMMLKM2').getOrCreate()

In [0]:
df = spark.read.csv('dbfs:/FileStore/tables/incident_event_log_reduced.csv',inferSchema=True,header=True)

------------------------------------------------------------------------------------------------------------------------------

##### Data pre-processing

In [0]:
# The data set has multiple states(New, Active, Awaiting user info, Resolved, Closed etc. ) of an incident. With the help 
# of the below command, we are just filtering one record per incident, that has the truly closed state of the incident. 

df_unique_incidents=df.filter("incident_state=='Closed'").sort("sys_mod_count",ascending=False).dropDuplicates(["number"])

In [0]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to make predictions

data=df_unique_incidents.select(['opened_by','location','category','subcategory',
                                 'u_symptom','assignment_group','priority'])

In [0]:
data=data.dropna()

------------------------------------------------------------------------------------------------------------------------------

### Building the K Means model

In [0]:
# Import the required libraries

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [0]:
# Use StringIndexer to convert the categorical columns to hold numerical data

opened_by_indexer = StringIndexer(inputCol='opened_by',outputCol='opened_by_index',handleInvalid='keep')
location_indexer = StringIndexer(inputCol='location',outputCol='location_index',handleInvalid='keep')
category_indexer = StringIndexer(inputCol='category',outputCol='category_index',handleInvalid='keep')
subcategory_indexer = StringIndexer(inputCol='subcategory',outputCol='subcategory_index',handleInvalid='keep')
u_symptom_indexer = StringIndexer(inputCol='u_symptom',outputCol='u_symptom_index',handleInvalid='keep')
assignment_group_indexer = StringIndexer(inputCol='assignment_group',outputCol='assignment_group_index',handleInvalid='keep')
priority_indexer = StringIndexer(inputCol='priority',outputCol='priority_index',handleInvalid='keep')

In [0]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['opened_by_index','location_index','category_index',
                                       'subcategory_index','u_symptom_index','assignment_group_index'],
                            outputCol="features")

In [0]:
# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data.

pipe = Pipeline(stages=[opened_by_indexer,location_indexer,category_indexer,subcategory_indexer,
                        u_symptom_indexer,assignment_group_indexer,priority_indexer,assembler])

In [0]:
# It took 5 minutes for this step to complete execution
final_data=pipe.fit(data).transform(data)

In [0]:
# Create an object for the Logistic Regression model

kmeans_model = KMeans(k=4)

In [0]:
fit_model = kmeans_model.fit(final_data)

In [0]:
wssse = fit_model.computeCost(final_data)
print("The within set sum of squared error of the mode is {}".format(wssse))

The within set sum of squared error of the mode is 37263960.473435

In [0]:
# # Make predictions
predictions = fit_model.transform(final_data)
from pyspark.ml.evaluation import ClusteringEvaluator
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))


Silhouette with squared euclidean distance = 0.7012985123003945

In [0]:

# # Shows the result.
# centers = kmeans_model.clusterCenters()

# print("Cluster Centers")
# index=1
# for cluster in centers:
#     print("Centroid {}: {}".format(index,cluster))
#     index+=1

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-699501117654528> in <module> 
 1 
 2 # Shows the result. 
 ----> 3 centers = kmeans_model . clusterCenters ( ) 
 4 
 5 print ( "Cluster Centers" ) 

 AttributeError : 'KMeans' object has no attribute 'clusterCenters'

In [0]:
# Store the results in a dataframe
results = fit_model.transform(final_data)

In [0]:
results.select(['opened_by_index','location_index','category_index','subcategory_index',
                'u_symptom_index','assignment_group_index','prediction']).show()

+---------------+--------------+--------------+-----------------+---------------+----------------------+----------+
opened_by_index|location_index|category_index|subcategory_index|u_symptom_index|assignment_group_index|prediction|
+---------------+--------------+--------------+-----------------+---------------+----------------------+----------+
 14.0| 4.0| 4.0| 4.0| 1.0| 0.0| 2|
 9.0| 4.0| 1.0| 2.0| 1.0| 0.0| 2|
 2.0| 0.0| 10.0| 3.0| 41.0| 7.0| 2|
 3.0| 5.0| 8.0| 11.0| 1.0| 11.0| 2|
 29.0| 22.0| 10.0| 3.0| 12.0| 7.0| 2|
 17.0| 18.0| 5.0| 3.0| 1.0| 18.0| 2|
 29.0| 2.0| 2.0| 58.0| 24.0| 14.0| 0|
 17.0| 1.0| 7.0| 14.0| 1.0| 46.0| 2|
 13.0| 16.0| 5.0| 3.0| 171.0| 12.0| 1|
 6.0| 0.0| 1.0| 2.0| 1.0| 0.0| 2|
 10.0| 1.0| 22.0| 32.0| 4.0| 4.0| 2|
 13.0| 0.0| 3.0| 40.0| 55.0| 19.0| 0|
 9.0| 0.0| 3.0| 0.0| 1.0| 0.0| 2|
 0.0| 0.0| 6.0| 6.0| 19.0| 1.0| 2|
 0.0| 29.0| 18.0| 0.0| 1.0| 30.0| 2|
 0.0| 9.0| 4.0| 0.0| 0.0| 1.0| 2|
 16.0| 5.0| 24.0| 75.0| 44.0| 36.0| 0|
 16.0| 0.0| 3.0| 1.0| 27.0| 3.0| 2|
 6.0| 0.0| 1.0| 5.0| 20.0| 1.0| 2|
 15.0| 2.0| 1.0| 2.0| 29.0| 0.0| 2|
+---------------+--------------+--------------+-----------------+---------------+----------------------+----------+
only showing top 20 rows

In [0]:
results.groupby('prediction').count().sort('prediction').show()

+----------+-----+
prediction|count|
+----------+-----+
 0| 2480|
 1| 1092|
 2|19886|
 3| 1460|
+----------+-----+

-------------------------------------------------------------------------------------------------------------------------------